In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import wandb

import numpy as np
import seaborn as sns
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

import sys
BASE_PATH = globals()['_dh'][0].parent.absolute()
sys.path.insert(1, str(BASE_PATH))

from src.utils.utils import count_parameters
from src.models.models import GCN, GAT
from src.models.iterativeModels import iterativeGCN, iterativeGAT
from src.models.variantModels import iterativeGCN_variant
from src.utils.wandb_analysis import get_metrics, get_clean_sweep_runs, get_sweep_info
wandb.login()

True

In [7]:
dataset = Planetoid(root='data/Planetoid/',
                     name='Cora',
                     transform=NormalizeFeatures())
num_features, num_classes = dataset.num_features, dataset.num_classes

In [ ]:
gcn = GCN(input_dim=num_features,
          output_dim=num_classes,
          hidden_dim=32,
          num_layers=4,
          dropout=0.5)
count_parameters(gcn)

In [ ]:
igcn = iterativeGCN(input_dim=num_features,
                    output_dim=num_classes,
                    hidden_dim=32,
                    dropout=0.5,
                    train_schedule=None
)
count_parameters(igcn)

In [76]:
gat = GAT(num_node_features=num_features, 
          hidden_dim=8,
          output_dim=num_classes,
          num_layers=2,
          attn_dropout_rate=0.6,
          dropout=0.6,
          heads=8)
print("number of parameters: {}".format(count_parameters(gat)))
for p in gat.parameters():
    print("Size:{}, {}".format(p.size(), p.size().numel()))

number of parameters: 92373
Size:torch.Size([1, 8, 8]), 64
Size:torch.Size([1, 8, 8]), 64
Size:torch.Size([64]), 64
Size:torch.Size([64, 1433]), 91712
Size:torch.Size([1, 1, 7]), 7
Size:torch.Size([1, 1, 7]), 7
Size:torch.Size([7]), 7
Size:torch.Size([7, 64]), 448


In [78]:
igat = iterativeGAT(input_dim=num_features,
                    output_dim=num_classes,
                    hidden_dim=64,
                    heads=8,
                    attn_dropout_rate=0.6,
                    dropout=0.6,
                    train_schedule=None)
print("number of parameters: {}".format(count_parameters(igat)))
for p in igat.parameters():
    print("Size:{}, {}".format(p.size(), p.size().numel()))

number of parameters: 126087
Size:torch.Size([64, 1433]), 91712
Size:torch.Size([64]), 64
Size:torch.Size([1, 8, 64]), 512
Size:torch.Size([1, 8, 64]), 512
Size:torch.Size([64]), 64
Size:torch.Size([512, 64]), 32768
Size:torch.Size([7, 64]), 448
Size:torch.Size([7]), 7


In [68]:
from torch_geometric.nn import GATConv
dataset = Planetoid(root='data/Planetoid/',
                     name='Cora',
                     transform=NormalizeFeatures())
data = dataset[0]
gc = GATConv(in_channels=1433,
             out_channels=9,
             heads=10,
             concat=True
             )
count_parameters(gc)
for p in gc.parameters():
    if p.requires_grad:
        print(p.size())

torch.Size([1, 10, 9])
torch.Size([1, 10, 9])
torch.Size([90])
torch.Size([90, 1433])


In [48]:
igcnv = iterativeGCN_variant(input_dim=num_features,
                             output_dim=num_classes,
                             hidden_dim=32,
                             train_schedule=None,
                             dropout=0.5)
count_parameters(igcnv)

17187

In [79]:
from src.utils.metrics import MAD
from src.models.iterativeModels import iterativeGCN_inductive
from src.utils.utils import exp_mol, make_uniform_schedule
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader

dataset = PygGraphPropPredDataset(name='ogbg-molhiv') 
split_idx = dataset.get_idx_split() 
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)
evaluator = Evaluator(name="ogbg-molhiv")


In [80]:
from src.models.models import GCN_inductive
from src.utils.utils import count_parameters
model = GCN_inductive(
            num_tasks=dataset.num_tasks,
            hidden_dim=300,
            num_layers=5,
            dropout=0.5)
count_parameters(model)

507001

In [92]:
from src.models.iterativeModels import iterativeGCN_inductive
model_i = iterativeGCN_inductive(
    num_tasks=dataset.num_tasks,
    hidden_dim=628,
    train_schedule=None
)
count_parameters(model_i)

506169